In [ ]:
from confluent_kafka import Consumer, KafkaError
from lightning import Lightning
from numpy import random
from numpy import concatenate
from numpy import array
from numpy import empty
from numpy import append
from numpy import around
from numpy import log
from numpy import fabs
import time

outliers_x,outliers_y,data_points_x,data_points_y,cl_x,cl_y = ([],[],[],[],[],[])
x_n,y_n=(array([], dtype=float), array([], dtype=float))
centroid_data_x = centroid_data_y = {}

centroid_records_seen = 0
total_records_seen = 0

first_centroids_record_passed = False

# max number of points per cluster to show (we use this for practical reasons)
MAX_DATA_POINTS_PER_CLUSTER = 25

MAX_OUTLIERS_TO_SHOW = 25

# blue color for centroids
centroid_color = [0, 0, 255]

# green color for points
point_color = [0, 255, 0]

# red color for outlier points
outlier_color = [255, 0, 0]

cluster_topic="nwcls"
brokers="broker.kafka.l4lb.thisdcos.directory:9092"

lgn = Lightning()

description = '''
<div style="text-align:center">
<span style="color: black; font-weight: bold;">Network Intrusion App - Visualization </span> </br>
<span style="color: orange; font-weight: bold;">FDP Platform - Lightbend </span>
</div>'''

viz=None

def str_to_bool(s):
    if s == 'True':
         return True
    elif s == 'False':
         return False
    else:
         raise ValueError
        
def parse_msg (msg):
  global viz
  global data_points_x,data_points_y,cl_x,cl_y
  global x_n
  global y_n
  global centroid_data_x, centroid_data_y
  global centroid_records_seen
  global total_records_seen
  global first_centroids_record_passed

  isOutlier = False
  total_records_seen = total_records_seen + 1

  data_points_x,data_points_y = ([],[])

  if  msg.find("Centroids:/") >= 0 :
    #print(msg)
    centroid_records_seen =  centroid_records_seen + 1
    splitted=msg.strip("Centroids:/").split("/")
    data_size=len(splitted)

    # new clusters arrived remove old data
    cl_x,cl_y = ([],[])
    centroid_data_x = centroid_data_y = {}
    x_n,y_n = (array([], dtype=float), array([], dtype=float))

    for cluster_center_point in splitted:
      if not first_centroids_record_passed: # clean old data for all cases but the first centroid record
        centroid_data_x[cluster_center_point] = []
        centroid_data_y[cluster_center_point] = []
      else : # It might be the case that we received data first
        centroid_data_x.setdefault(cluster_center_point, [])
        centroid_data_y.setdefault(cluster_center_point, [])
      pair = cluster_center_point.split(",")
      cl_x.append(float(pair[0]))
      cl_y.append(float(pair[1]))
    
    if not first_centroids_record_passed:
        first_centroids_record_passed = True 
        
  else :
    splitted = msg.split(",")
    cluster_center_point = splitted[1].strip("[") + "," + splitted[2].strip("]")
    x_1 = splitted[-2] # new point x coordinate 
    y_1 = splitted[-1] # new point y coordinate
    isOutlier = str_to_bool(splitted[5].capitalize())
    
    if isOutlier:
        points_for_update_x = outliers_x
        points_for_update_y = outliers_y
    else: 
        if first_centroids_record_passed: # we might be the first to update the map
            centroid_data_x.setdefault(cluster_center_point, [])
            centroid_data_y.setdefault(cluster_center_point, [])
        points_for_update_x = centroid_data_x[cluster_center_point]
        points_for_update_y = centroid_data_y[cluster_center_point]

    if len(points_for_update_x) <= MAX_DATA_POINTS_PER_CLUSTER:
        points_for_update_x.append(float(x_1))
        points_for_update_y.append(float(y_1))
    else:
        record_pointer = (total_records_seen - centroid_records_seen) % MAX_DATA_POINTS_PER_CLUSTER
        points_for_update_x[record_pointer] = float(x_1)
        points_for_update_y[record_pointer] = float(y_1)
      
    
  for value in centroid_data_x.values():
    data_points_x = data_points_x + value
    
  for value in centroid_data_y.values():
    data_points_y = data_points_y + value

  cl_n_x = array(cl_x)
  cl_n_y = array(cl_y)

  data_n_x = array(data_points_x)
  data_n_y = array(data_points_y)
    
  outliers_n_x = array(outliers_x)
  outliers_n_y = array(outliers_y)

  x_n = concatenate((cl_n_x, data_n_x))
  x_n = concatenate((x_n, outliers_n_x))
  y_n = concatenate((cl_n_y, data_n_y))
  y_n = concatenate((y_n, outliers_n_y))

  # start visualization here, it does not work if we start with empty data
  if viz == None:
    viz = lgn.scatterstreaming(x_n, y_n, description = description)
    vis_link = viz.get_permalink()
    public_ip_list = !curl http://169.254.169.254/latest/meta-data/public-ipv4
    host_public_ip=public_ip_list.l[-1]
    print("Click here to view streaming k-means: " + vis_link.replace('localhost', host_public_ip))
    
  data_colors = empty((len(data_points_x),),dtype=object)
  cluster_center_colors = empty((len(cl_x),),dtype=object)
  outliers_colors = empty((len(outliers_x),),dtype=object)

  data_colors.fill(point_color)
  cluster_center_colors.fill(centroid_color)
  outliers_colors.fill(outlier_color)

  color_list = concatenate((cluster_center_colors, data_colors))
  color_list = concatenate((color_list, outliers_colors)).tolist()
  viz.append(x_n, y_n, color = color_list)
  return

# Make sure we restart when we re-run the code for the same group id
def on_assign (c, ps):
    for p in ps:
        print("Topic info..." + p)
        p.offset=-2
        c.assign(ps)

# Create a consumer
rand_suffix = str(int(round(time.time() * 1000)))
c = Consumer({'bootstrap.servers': brokers, 'group.id': 'consume-nwcin-' + rand_suffix,
                'default.topic.config': {'auto.offset.reset': 'earliest'}})

# subscribe to the cluster topic. We always start from the beginning
# even if we re-use the same group.id
c.subscribe([cluster_topic])

# Loop until we have an error, blocks if no messages are left
counter=0
running = True
while running:

    start = time.time()
    msg = c.poll()

    if not msg.error():
        #print('Received message: %s' % msg.value().decode('utf-8'))
        counter = counter  + 1
        #if counter % 10000 == 0:
        #    print("Records processed..." + str(counter))
        parse_msg(msg.value().decode('utf-8'))
    elif msg.error().code() != KafkaError._PARTITION_EOF:
        print(msg.error())
        running = False
c.close()
